In [4]:
import torch
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer, pipeline, CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import requests

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from g_spread import *

gsp = GSpread()
links = gsp.getAllLinks()

In [6]:
df = pd.DataFrame(links, columns=['link'])
df.head()

,link
0,https://cdn.pixabay.com/photo/2023/02/08/07/32...
1,https://cdn.pixabay.com/photo/2023/01/27/06/17...
2,https://cdn.pixabay.com/photo/2023/02/09/16/36...
3,https://cdn.pixabay.com/photo/2023/02/11/13/43...
4,https://firebasestorage.googleapis.com/v0/b/si...


In [4]:
image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

df['captioning'] = df.apply(lambda x: image_to_text(x.link)[0]['generated_text'], axis = 1)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [5]:
df.head(5)

,link,captioning
0,https://cdn.pixabay.com/photo/2023/02/08/07/32...,a woman dressed in a costume holding a white bow
1,https://cdn.pixabay.com/photo/2023/01/27/06/17...,a brown and white bird standing in a field
2,https://cdn.pixabay.com/photo/2023/02/09/16/36...,a bridge with a large group of people on it
3,https://cdn.pixabay.com/photo/2023/02/11/13/43...,a lake with trees and a bench
4,https://firebasestorage.googleapis.com/v0/b/si...,a large advertisement for a television show


In [7]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [ ]:
def classifier1(url, label=['real photo', 'document']):
    image = Image.open(requests.get(url, stream=True).raw)
    inputs = processor(text=label, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1) #
    return probs# torch.max(probs, dim=1).indices.item()

In [9]:
df['class1'] = df.link.apply(lambda x: classifier1(x, label=['picture', 'instagram', 'only']))
df.head(20)

: 

: 

In [18]:
gsp.updateByDf(df[['captioning', 'class1']], [4,3])